# AN2DL - Second Challenge

Lorenzo Bardelli, Lorenzo Moretti, Luca Zani

⁠`thegradientdescenders`

# Preprocessing

Clean the dataset from unwanted samples of shrek and green blobs and augment it by creating "patches", each image in the dataset will be used to extract patches; those will allow us to have more training data to work with

Where in the image to extract a patch is determined by the mask attached to the image

In [ ]:
SEED = 123

import os
import random
import numpy as np
import pandas as pd
import glob
import psutil
from concurrent.futures import ThreadPoolExecutor

np.random.seed(SEED)
random.seed(SEED)

import cv2
import matplotlib.pyplot as plt
import seaborn as sns

current_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
dataset_dir = os.path.join(current_dir, "dataset")
print(f"Dataset directory: {dataset_dir}")

os.makedirs(os.path.join(current_dir, "preprocessed"), exist_ok=True)
SAVE_DIR = os.path.join(current_dir, 'preprocessed')

os.makedirs(os.path.join(SAVE_DIR, "train_data"), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, "test_data"), exist_ok=True)


# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

In [ ]:
# Hardcoded bad indices, detected via a python script but also added manually in some cases
BAD_INDICES = {
    1,
    5,
    8,
    12,
    18,
    20,
    22,
    27,
    28,
    36,
    44,
    47,
    48,
    52,
    62,
    78,
    85,
    90,
    94,
    95,
    129,
    130,
    133,
    136,
    138,
    148,
    150,
    155,
    159,
    161,
    175,
    178,
    179,
    180,
    184,
    187,
    189,
    193,
    196,
    222,
    251,
    254,
    263,
    268,
    286,
    293,
    313,
    319,
    333,
    342,
    344,
    346,
    355,
    368,
    371,
    376,
    380,
    390,
    393,
    407,
    410,
    415,
    424,
    443,
    453,
    459,
    463,
    486,
    497,
    498,
    499,
    509,
    521,
    530,
    531,
    533,
    537,
    540,
    544,
    547,
    557,
    558,
    560,
    565,
    567,
    572,
    578,
    580,
    586,
    602,
    603,
    607,
    609,
    614,
    620,
    623,
    629,
    635,
    639,
    643,
    644,
    645,
    646,
    656,
    657,
    658,
    670,
    673,
    675,
}

In [ ]:
def save_patches(id, patches, data_type='train'):
    """Save patches to appropriate subfolder.
    
    Args:
        id: Image ID
        patches: List of patch arrays
        data_type: 'train' or 'test' to specify which subfolder to use
    """
    if SAVE_DIR:
        save_path = os.path.join(SAVE_DIR, f"{data_type}_data")
        os.makedirs(save_path, exist_ok=True)
        
        for count, img_patch in enumerate(patches):
            # Convert float32 (0-1) to uint8 (0-255) for saving
            img_patch_uint8 = (img_patch * 255).astype(np.uint8)
            # Format: img_XXXX_patch_YY.png where XXXX is image ID (4 digits), YY is patch number (2 digits)
            cv2.imwrite(os.path.join(save_path, f"img_{id:04d}_patch_{count:02d}.png"), img_patch_uint8)

# Function to load images and masks from a folder with parallel loading
def load_patches_from_folder(folder, use_parallel=True, max_workers=8, patch_size=128, stride=64, max_patches=12, data_type='train', filter_bad_indices=True):
    """Load and preprocess images and masks from a specified folder with parallel loading.
    
    Args:
        folder: Path to the folder containing images
        use_parallel: Whether to use parallel processing
        max_workers: Number of parallel workers
        patch_size: Size of patches to extract
        stride: Stride for patch extraction
        max_patches: Maximum number of patches per image
        data_type: 'train' or 'test' - determines subfolder for saving and whether to filter bad indices
        filter_bad_indices: If True, skip images in blacklist (only applies to 'train' data)
    
    Returns:
        tuple: (patches, indices) where patches is a list of np.arrays and indices is a list of image indices
    """

    def make_patches(image, mask, img_id=None):
        """Extracts patches from image and mask, keeping those with significant mask content."""
        
        H, W = mask.shape[:2]

        top_patches = []

        count = 0
        for y in range(0, H - patch_size + 1, stride):
            for x in range(0, W - patch_size + 1, stride):
                img_patch = image[y:y+patch_size, x:x+patch_size]
                mask_patch = mask[y:y+patch_size, x:x+patch_size]

                # Check importance
                mask_ratio = np.count_nonzero(mask_patch) / (patch_size * patch_size)
                if mask_ratio > 0:
                    top_patches.append( (img_patch, mask_ratio) )
                    count += 1

        return [x[0] for x in sorted(top_patches, key=lambda x: x[1], reverse=True)[:max_patches]]
    
    def print_memory():
        print(f"RAM used: {psutil.Process().memory_info().rss / 1024 / 1024:.0f} MB")
    
    def process_single_image(filename):
        """Process a single image-mask pair. Returns (idx, patches) or None."""
        # Extract index from filename (e.g., img_0000.png -> 0)
        idx = int(filename.split('_')[1].split('.')[0])
        
        # Only filter bad indices for training data
        if filter_bad_indices and data_type == 'train' and idx in BAD_INDICES:
            return None
        
        # Load image and mask
        img = cv2.imread(os.path.join(folder, filename))
        mask = cv2.imread(os.path.join(folder, filename.replace('img_', 'mask_')), cv2.IMREAD_GRAYSCALE)

        if img is None or mask is None:
            return None

        # Convert to float32 and normalize BEFORE padding (for filtering)
        img_original = img.astype(np.float32) / 255.0
        
        # Convert mask to binary (single channel): 0 or 1
        mask_binary = (mask > 127).astype(np.float32)  # Set threshold at 127, value > 127 -> 1, else 0
        mask_binary = np.expand_dims(mask_binary, axis=-1)[:,:,0]  # Add channel dimension (H, W, 1)

        patches = make_patches(img_original, mask_binary, img_id=idx)

        # Save patches to appropriate subfolder based on data type
        save_patches(idx, patches, data_type=data_type)
        return (idx, patches)
    
    # Get sorted list of image files (not masks) - using glob is faster
    filenames = sorted(glob.glob(os.path.join(folder, 'img_*.png')), key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
    filenames = [os.path.basename(f) for f in filenames]
    
    if not filenames:
        raise ValueError(f"No image files found in {folder}")
    
    print(f"Loading {len(filenames)} images from {folder}...")
    
    print_memory()

    if use_parallel and len(filenames) > 10:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(executor.map(process_single_image, filenames))
    else:
        results = [process_single_image(f) for f in filenames]
    
    # Filter out None results and unpack (idx, patches) tuples
    # Results from executor.map are in the same order as input filenames
    valid_results = [r for r in results if r is not None]
    indices = [r[0] for r in valid_results]
    patches = [np.array(r[1], dtype=np.float32) for r in valid_results]
    return patches, indices

In [ ]:
# Load training data (with blacklist filtering)
print("\n=== Processing Training Data ===")
train_patches, imageIndeces = load_patches_from_folder(
    dataset_dir + '/' + 'train_data', 
    patch_size=128, 
    data_type='train',
    filter_bad_indices=True
)

# Load test data (without blacklist filtering)
print("\n=== Processing Test Data ===")
test_patches, test_imageIndeces = load_patches_from_folder(
    dataset_dir + '/' + 'test_data', 
    patch_size=128, 
    data_type='test',
    filter_bad_indices=False
)

# Load training labels
labels_df = pd.read_csv(os.path.join(dataset_dir, 'train_labels.csv'))

print(f"\n=== Summary ===")
print(f"Loaded {len(train_patches)} training images")
print(f"Training image indices: {imageIndeces[:10]} ...")
print(f"Loaded {len(test_patches)} test images")
print(f"Test image indices: {test_imageIndeces[:10]} ...")

print(f"\nLabel distribution:")
print(labels_df['label'].value_counts())


In [ ]:
# Visualize original images, masks, and extracted patches

def display_images(num_images=10, max_patches_display=6):
    """Display original images with mask overlay and their extracted patches."""
    
    folder = os.path.join(dataset_dir, 'train_data')
    
    fig, axes = plt.subplots(num_images, max_patches_display + 2, figsize=(20, 3 * num_images))
    
    for i in range(min(num_images, len(train_patches))):
        idx = random.randint(0, len(train_patches) - 1)
        actual_img_id = imageIndeces[idx]  # Get the actual image ID

        # Use zero-padded filename format (img_0000.png, img_0001.png, etc.)
        img_path = os.path.join(folder, f'img_{actual_img_id:04d}.png')
        mask_path = os.path.join(folder, f'mask_{actual_img_id:04d}.png')
        
        # Load original image
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        
        # Load mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask_binary = (mask > 127).astype(np.float32)
        
        # Show original image
        axes[i, 0].imshow(img_rgb)
        axes[i, 0].set_title(f"Original img_{actual_img_id:04d}")
        axes[i, 0].axis('off')
        
        # Show image with mask overlay (red tint on masked regions)
        overlay = img_rgb.copy()
        overlay[mask_binary > 0.5] = overlay[mask_binary > 0.5] * 0.5 + np.array([1.0, 0.0, 0.0]) * 0.5
        axes[i, 1].imshow(np.clip(overlay, 0, 1))
        axes[i, 1].set_title(f"Mask overlay")
        axes[i, 1].axis('off')
        
        # Show patches
        patches = train_patches[idx]
        for j in range(max_patches_display):
            ax = axes[i, j + 2]
            if j < len(patches):
                patch_rgb = cv2.cvtColor((patches[j] * 255).astype(np.uint8), cv2.COLOR_BGR2RGB) / 255.0
                ax.imshow(patch_rgb)
                ax.set_title(f"Patch {j}")
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

display_images()

In [ ]:
# Copy train_labels.csv from dataset to preprocessed folder
import shutil

src_labels_path = os.path.join(dataset_dir, 'train_labels.csv')
dst_labels_path = os.path.join(SAVE_DIR, 'train_labels.csv')

shutil.copy(src_labels_path, dst_labels_path)

print(f"Copied {src_labels_path} -> {dst_labels_path}")